In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Charger les données
df = pd.read_parquet("../.data/processed/user_features.parquet")

# Étape 1 – Créer une colonne total_events
df["total_events"] = df["count_view"] + df["count_cart"] + df["count_purchase"]

# Étape 2 – Visualiser la distribution
sns.histplot(df["total_events"], bins=100)
plt.yscale("log")
plt.title("Distribution du total d'événements par utilisateur")
plt.xlabel("Total événements (vues + paniers + achats)")
plt.ylabel("Nombre d'utilisateurs (log)")
plt.show()

# Étape 3 – Appliquer un seuil (ici ≥ 5)
filtered_df = df[df["total_events"] >= 5]
print(f"{len(filtered_df):,} utilisateurs retenus sur {len(df):,} après filtrage")

In [2]:
filtered_df = df[df["total_events"] >= 5]
print(f"{len(filtered_df):,} utilisateurs retenus sur {len(df):,} après filtrage")

7,841,232 utilisateurs retenus sur 15,639,803 après filtrage


In [4]:
filtered_df.to_parquet(
    "../.data/processed/filtered_user_features.parquet", index=False)

In [2]:
filtered_df = pd.read_parquet(
    "../.data/processed/filtered_user_features.parquet")

In [3]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Liste des variables retenues
features = [
    "count_view", "count_cart", "count_purchase",
    "unique_sessions", "active_days", "recency_days",
    "total_spent", "avg_purchase_price"
]

# Filtrage des utilisateurs actifs déjà effectué dans filtered_df
X = filtered_df[features].copy()

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7841232 entries, 2 to 15639783
Data columns (total 8 columns):
 #   Column              Dtype  
---  ------              -----  
 0   count_view          int64  
 1   count_cart          int64  
 2   count_purchase      int64  
 3   unique_sessions     int64  
 4   active_days         int64  
 5   recency_days        int64  
 6   total_spent         float64
 7   avg_purchase_price  float64
dtypes: float64(2), int64(6)
memory usage: 538.4 MB


In [7]:
X.describe()

,count_view,count_cart,count_purchase,unique_sessions,active_days,recency_days,total_spent,avg_purchase_price
count,7.841232e+06,7.841232e+06,7.841232e+06,7.841232e+06,7.841232e+06,7.841232e+06,7.841232e+06,7.841232e+06
mean,4.733847e+01,2.404705e+00,8.621257e-01,1.020169e+01,6.472449e+00,2.860016e+00,2.586070e+02,3.635623e+01
std,1.501978e+02,7.862739e+00,4.427712e+00,7.925090e+01,8.551692e+00,3.678121e+00,1.937913e+03,1.181796e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.700000e+01,0.000000e+00,0.000000e+00,5.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,4.400000e+01,2.000000e+00,1.000000e+00,1.100000e+01,7.000000e+00,4.000000e+00,9.130000e+00,1.651429e+00
max,1.991750e+05,2.342000e+03,2.120000e+03,1.306690e+05,2.170000e+02,2.100000e+01,7.901209e+05,2.574070e+03


In [4]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Copie de X pour transformation
X_transformed = X.copy()

# Variables à transformer avec log(1 + x)
log_features = ["count_view", "count_cart",
                "count_purchase", "total_spent", "avg_purchase_price"]
X_transformed[log_features] = X_transformed[log_features].apply(
    lambda x: np.log1p(x))

# Standardisation finale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_transformed)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

inertias = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    logger.info(f"Training KMeans with k={k}")
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    logger.info(
        f"k={k}: Inertia={inertias:.2f}, Silhouette={silhouette_scores:.4f}")

# Affichage de l'inertie (méthode du coude)
plt.figure(figsize=(10, 5))
plt.plot(k_range, inertias, marker='o')
plt.title("Méthode du coude – Inertie en fonction du nombre de clusters")
plt.xlabel("Nombre de clusters (k)")
plt.ylabel("Inertie intra-cluster")
plt.grid(True)
plt.show()

# Affichage du score de silhouette
plt.figure(figsize=(10, 5))
plt.plot(k_range, silhouette_scores, marker='s', color='green')
plt.title("Score de silhouette moyen selon k")
plt.xlabel("Nombre de clusters (k)")
plt.ylabel("Silhouette moyenne")
plt.grid(True)
plt.show()

INFO:__main__:Training KMeans with k=2


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# 1. Prédiction des clusters pour tout le dataset (ou un sous-échantillon)
filtered_df["cluster"] = kmeans.predict(X_scaled)

# 2. Moyenne des variables par cluster (profilage)
cluster_profile = filtered_df.groupby("cluster")[
    ["count_view", "count_cart", "count_purchase",
     "total_spent", "avg_purchase_price",
     "active_days", "recency_days"]
].mean().round(1)

# 3. Affichage de la heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(cluster_profile, annot=True, fmt=".1f", cmap="YlGnBu")
plt.title("Profil moyen des utilisateurs par cluster")
plt.xlabel("Variables comportementales")
plt.ylabel("Clusters")
plt.tight_layout()
plt.show()